#### import the things

In [1]:
import pandas as pd
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

idx = pd.IndexSlice

import scipy.stats

In [14]:
with open(r"..\path.txt") as p:
    path = p.readlines()[0]
import sys
sys.path.append(path)

import data_sorting_tools as dst

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Understanding the key layout

In [4]:
raw_data_fol = r"..\data_files"

In [5]:
query1_file = os.path.join(raw_data_fol, "main_search1.json")
with open(query1_file) as f:
    query1 = json.load(f)

In [7]:
query1['1'].keys()

dict_keys(['created_at', 'id', 'id_str', 'text', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'retweeted_status', 'is_quote_status', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'entities', 'favorited', 'retweeted', 'filter_level', 'lang', 'matching_rules'])

In [10]:
for n in query1.keys():
    if query1[n]['truncated']==True:
        print(n)

3
15
18
27
28
34
37
38
59
62
68
76
78
80
86
89
91
93
97
98
113
114
120
125
126
131
135
146
157
159
166
169
172
174
175
176
177
182
193
199
205
206
215
219
223
229
230
231
234
238
241
246
249
250
251
262
264
268
269
272
273
276
280
283
294
297
298
303
310
311
314
315
334
342
354
362
374
376
378
379
385
386
391
392
394
396
403
420
422
429
431
434
435
440
443
445
448
450
452
455
458
463
464
468
472
473
481
488
493


TypeError: string indices must be integers

In [11]:
query1['3'].keys()

dict_keys(['created_at', 'id', 'id_str', 'text', 'display_text_range', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'extended_tweet', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'entities', 'favorited', 'retweeted', 'possibly_sensitive', 'filter_level', 'lang', 'matching_rules', 'extended_text'])

#### create dataframe from main dataset

In [12]:
keep_keys = ['quote_count', 'reply_count', 'retweet_count', 'favorite_count', ('user','id'), 
                ('user','followers_count'), ('user','verified'), 'text', 'lang']

In [80]:
all_dfs = []
for f in os.listdir(raw_data_fol):
    if ("main" in f) and (".json" in f):
        d = dst.json_to_df(os.path.join(raw_data_fol, f), keep_keys, get_media=True, return_dict=False)
        all_dfs.append(d)
#         print(f, len(d))
        
df = pd.concat(all_dfs)

In [81]:
df.head()

,quote_count,reply_count,retweet_count,favorite_count,user_id,user_followers_count,user_verified,extended_text,text,lang,media_count,media_types
1376413638995550209,52,87,298,1140,1398479138,215624,True,Also well known that after his HIV diagnosis h...,Also well known that after his HIV diagnosis h...,en,0,none
1376424348173668353,1,2,19,45,188289863,22153,False,New study on HIV testing and trans people find...,New study on HIV testing and trans people find...,en,0,none
1376424031965036545,0,0,5,2,1195310926976765952,4776,False,"New in @TheLancetPH&gt; Homelessness, unstable...","New in @TheLancetPH&gt; Homelessness, unstable...",en,0,none
1374250344134369281,22,36,568,1726,129009730,546051,False,NaN,Liberia's Dr. Dougbeh Chris Nyan invented a ra...,en,0,none
1375010567136546816,52,12,439,481,1103617280926191616,2773,False,NaN,Today I learnt that @utm_my (one of Malaysia's...,en,0,none


In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11371 entries, 1376413638995550209 to 1375441242121764864
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   quote_count           11371 non-null  object
 1   reply_count           11371 non-null  object
 2   retweet_count         11371 non-null  object
 3   favorite_count        11371 non-null  object
 4   user_id               11371 non-null  object
 5   user_followers_count  11371 non-null  object
 6   user_verified         11371 non-null  object
 7   extended_text         4826 non-null   object
 8   text                  11371 non-null  object
 9   lang                  11371 non-null  object
 10  media_count           11371 non-null  int64 
 11  media_types           11371 non-null  object
dtypes: int64(1), object(11)
memory usage: 1.1+ MB


In [83]:
text_compare = df[pd.isnull(df.extended_text)==False][['text', 'extended_text']]
text_compare

,text,extended_text
1376413638995550209,Also well known that after his HIV diagnosis h...,Also well known that after his HIV diagnosis h...
1376424348173668353,New study on HIV testing and trans people find...,New study on HIV testing and trans people find...
1376424031965036545,"New in @TheLancetPH&gt; Homelessness, unstable...","New in @TheLancetPH&gt; Homelessness, unstable..."
1376423839488430082,#TigrayGenocide \nNurse Mulu seen 260 rape vic...,#TigrayGenocide \nNurse Mulu seen 260 rape vic...
1376422998572290049,Nurse Mulu seen 260 rape victims which reporte...,Nurse Mulu seen 260 rape victims which reporte...
...,...,...
1375445184956223490,Get signing up to this great line up including...,Get signing up to this great line up including...
1375445344742440960,"@thehill The man who made this quote\n""It is t...","@thehill The man who made this quote\n""It is t..."
1375445284885565440,@Thulani82363945 @Thembisile_Q @MoshekiD @Sebu...,@Thulani82363945 @Thembisile_Q @MoshekiD @Sebu...
1375445180052992000,@sinoprideplus @petesmi39415195 @NewshubNZ HIV...,@sinoprideplus @petesmi39415195 @NewshubNZ HIV...


In [84]:
text_compare[text_compare.text == text_compare.extended_text]

,text,extended_text


In [85]:
text_compare.iloc[0].text

'Also well known that after his HIV diagnosis he continued to have sex with men in bath houses. (Keep in mind that t… https://t.co/MwgIWcdBht'

In [86]:
text_compare.iloc[0].extended_text

'Also well known that after his HIV diagnosis he continued to have sex with men in bath houses. (Keep in mind that this is the most cited scholar of all time) https://t.co/PyweojpuC0'

In [87]:
df['extended_text'].fillna(df['text'], inplace=True)

In [88]:
df.drop('text', inplace=True, axis=1)
df.rename(columns={'extended_text':'text'}, inplace=True)

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11371 entries, 1376413638995550209 to 1375441242121764864
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   quote_count           11371 non-null  object
 1   reply_count           11371 non-null  object
 2   retweet_count         11371 non-null  object
 3   favorite_count        11371 non-null  object
 4   user_id               11371 non-null  object
 5   user_followers_count  11371 non-null  object
 6   user_verified         11371 non-null  object
 7   text                  11371 non-null  object
 8   lang                  11371 non-null  object
 9   media_count           11371 non-null  int64 
 10  media_types           11371 non-null  object
dtypes: int64(1), object(10)
memory usage: 1.0+ MB


In [91]:
df.set_index('user_id', drop=True, append=True, inplace=True)
df.index.rename(['tid', 'uid'], inplace=True)

In [92]:
df.head()

,,quote_count,reply_count,retweet_count,favorite_count,user_followers_count,user_verified,text,lang,media_count,media_types
tid,uid,,,,,,,,,,
1376413638995550209,1398479138,52,87,298,1140,215624,True,Also well known that after his HIV diagnosis h...,en,0,none
1376424348173668353,188289863,1,2,19,45,22153,False,New study on HIV testing and trans people find...,en,0,none
1376424031965036545,1195310926976765952,0,0,5,2,4776,False,"New in @TheLancetPH&gt; Homelessness, unstable...",en,0,none
1374250344134369281,129009730,22,36,568,1726,546051,False,Liberia's Dr. Dougbeh Chris Nyan invented a ra...,en,0,none
1375010567136546816,1103617280926191616,52,12,439,481,2773,False,Today I learnt that @utm_my (one of Malaysia's...,en,0,none


### duplicates

first, drop total duplicates

In [99]:
df[df.index.duplicated(keep=False)].sort_index()

quote_count reply_count retweet_count  \
tid                 uid                                                         
925628690804875264  1063018452                   94          18          1715   
                    1063018452                   94          18          1715   
1028280350076592130 2741515593                    8           4           231   
                    2741515593                    8           4           231   
1062121966977437696 968217986845745152            1           5           188   
...                                             ...         ...           ...   
1376393859693957120 1354045976244322305           0           0             1   
1376394839491756033 3029911732                    1           6             7   
                    3029911732                    1           6             7   
1376397326663745544 2785063283                    0           0             1   
                    2785063283                    0           0             1   

                                        favorite_count user_followers_count  \
tid                 uid                                                       
925628690804875264  1063018452                    1880                 7544   
                    1063018452                    1880                 7544   
1028280350076592130 2741515593                     323                 1962   
                    2741515593                     324                 1962   
1062121966977437696 968217986845745152             555                26673   
...                                                ...                  ...   
1376393859693957120 1354045976244322305              3                   44   
1376394839491756033 3029911732                     138                19692   
                    3029911732                     138                19692   
1376397326663745544 2785063283                       2                10742   
                    2785063283                       2                10742   

                                        user_verified  \
tid                 uid                                 
925628690804875264  1063018452                  False   
                    1063018452                  False   
1028280350076592130 2741515593                  False   
                    2741515593                  False   
1062121966977437696 968217986845745152          False   
...                                               ...   
1376393859693957120 1354045976244322305         False   
1376394839491756033 3029911732                  False   
                    3029911732                  False   
1376397326663745544 2785063283                  False   
                    2785063283                  False   

                                                                                      text  \
tid                 uid                                                                      
925628690804875264  1063018452           HIV: A simple explanation.\n\nPlease RT and sp...   
                    1063018452           HIV: A simple explanation.\n\nPlease RT and sp...   
1028280350076592130 2741515593           HIV is transmitted ONLY through three ways:\n1...   
                    2741515593           HIV is transmitted ONLY through three ways:\n1...   
1062121966977437696 968217986845745152   Different angle..his moans were so hot!\n@Masc...   
...                                                                                    ...   
1376393859693957120 1354045976244322305  @amnestyusa I hope you are watching this too. ...   
1376394839491756033 3029911732           I Am not #HIV, I am Living with HIV...\nI am j...   
                    3029911732           I Am not #HIV, I am Living with HIV...\nI am j...   
1376397326663745544 2785063283           #Larkana continue to #top #HIV positive cases ...   
                    2785063283           #Larkana continue to #top #HIV positive cases .

In [103]:
duplicates = df[df.index.duplicated(keep='first')].index
df.shape

(11371, 10)

In [104]:
df = df.drop(index=duplicates)
df.shape

(8036, 10)

In [106]:
df[df.duplicated(subset='text', keep=False)].sort_values('text')

,,quote_count,reply_count,retweet_count,favorite_count,user_followers_count,user_verified,text,lang,media_count,media_types
tid,uid,,,,,,,,,,
1372261620605419522,20252263,0,0,0,0,7584,False,#Africa steps up fight against #HIV with trial...,en,0,none
1376323344694398976,20252263,0,0,0,0,7584,False,#Africa steps up fight against #HIV with trial...,en,0,none
1375496976209416192,1010757472238342145,0,0,0,1,191,False,#RT @pozmagazine: Thousands of Texans With HIV...,en,0,none
1376342591915106306,1010757472238342145,0,0,0,0,191,False,#RT @pozmagazine: Thousands of Texans With HIV...,en,0,none
1372330780362301442,1219715989874642947,0,2,0,0,426,False,@Cubbies108 @thedailybeast There is a differen...,en,0,none
...,...,...,...,...,...,...,...,...,...,...,...
1376323956010070018,1069209602624036864,0,0,0,3,3489,False,just passed my HIV test! bro i ain’t even had ...,en,0,none
1376323620084064258,4890404560,0,4,0,17,3582,False,just passed my HIV test! bro i ain’t even had ...,en,0,none
1374685059987337222,979357573773062144,0,0,0,1,1989,False,physician's advice on quitting smoking deliver...,en,0,none


In [107]:
more_duplicates = df[df.duplicated(subset='text', keep='first')].index
df.shape

(8036, 10)

In [108]:
df = df.drop(index=more_duplicates)
df.shape

(7978, 10)

check languages

In [109]:
df.lang.unique()

array(['en', 'da', 'ht', 'tl', 'nl', 'it'], dtype=object)

In [111]:
df[df.lang!='en'].text.values

array(['HIV Endgame : ‘Avengers’ #HVTN702, #Imbokodo and #Mosaico ?\n#HIV #vaccine \nhttps://t.co/eMgTuluNyx https://t.co/3ZNUlfd2pw',
       'P C by lol p hip pop  hiv g',
       'Proof na legit ang FREE HIV test! \nKnow your status! \nWag po mahiya, DM me. \n#alterbataan #alterbalanga https://t.co/ueaFIKDOdU',
       'Corned Beef, Sauerkraut and Thousand Island  ... \n🔪\n#MealPrep #ForkYeah #CornedBeef #StPatricksDay #FeedFeed #Prep #Delicious #Seasoning #NomNom #SmartFoodService @USFChefStore @LitehouseFoods @TysonFoods @McCormickSpices @SmartFinal @CrockPot #KrugerFoods #PleasantValleyFarm https://t.co/XQQ4M6oxlK',
       'HIV lol https://t.co/rUZOOTFO6e',
       '#HIV #aids #DATA: commento al #video (#FakeNEWS #scientifiche?)..(#brainstorming #ibse #inquire #storytelling)--   #educazione #civica #cittadinanza #scuole #medie #fake #news #dad #didattica a #distanza #digitale #riapertura &gt;&gt;   https://t.co/QGbdeLzWkG https://t.co/Bx2ssd7ZIF',
       'One day I will let the world

I feel comfortable dropping all of these tweets (I still really don't understand how they got through the tweepy language filter)

In [115]:
non_eng = df[df.lang!='en'].index
df.shape

(7978, 10)

In [116]:
df.drop(non_eng, inplace=True)
df.shape

(7971, 10)

In [117]:
df.groupby('tid').count().sort_values('quote_count', ascending=False)

,quote_count,reply_count,retweet_count,favorite_count,user_followers_count,user_verified,text,lang,media_count,media_types
tid,,,,,,,,,,
183749146224766977,1,1,1,1,1,1,1,1,1,1
1373481813339828226,1,1,1,1,1,1,1,1,1,1
1373487350626594817,1,1,1,1,1,1,1,1,1,1
1373486882101792778,1,1,1,1,1,1,1,1,1,1
1373486027143266308,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
1370283166745849858,1,1,1,1,1,1,1,1,1,1
1370272112439562244,1,1,1,1,1,1,1,1,1,1
1370269585006551040,1,1,1,1,1,1,1,1,1,1


In [122]:
df = df.convert_dtypes()

In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 7971 entries, (1376413638995550209, 1398479138) to (1375444543798140931, 1164152787824984064)
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   quote_count           7971 non-null   Int64  
 1   reply_count           7971 non-null   Int64  
 2   retweet_count         7971 non-null   Int64  
 3   favorite_count        7971 non-null   Int64  
 4   user_followers_count  7971 non-null   Int64  
 5   user_verified         7971 non-null   boolean
 6   text                  7971 non-null   string 
 7   lang                  7971 non-null   string 
 8   media_count           7971 non-null   Int64  
 9   media_types           7971 non-null   string 
dtypes: Int64(6), boolean(1), string(3)
memory usage: 1.4 MB


In [125]:
df.describe()

,quote_count,reply_count,retweet_count,favorite_count,user_followers_count,media_count
count,7971.000000,7971.000000,7971.000000,7971.000000,7.971000e+03,7971.000000
mean,1.231339,1.593276,15.015431,60.515494,2.254215e+04,0.035002
std,28.953529,20.235872,457.220487,1770.069682,2.840563e+05,0.222689
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
25%,0.000000,0.000000,0.000000,0.000000,1.410000e+02,0.000000
50%,0.000000,0.000000,0.000000,1.000000,7.080000e+02,0.000000
75%,0.000000,1.000000,1.000000,4.000000,2.845000e+03,0.000000
max,1877.000000,1314.000000,37622.000000,134053.000000,1.643666e+07,4.000000


Again I'm left with ~8000 tweets, from a pull of 20,000. Not the best returns. I can do one more data pull after this one.

In [127]:
#save the dataframe as a pickle file to be loaded into other notebooks
pd.to_pickle(df, os.path.join(raw_data_fol, "full_dataset_cleaned_pull2.pkl"))